# ES Active Orders - Product Mapping Export

This notebook exports active Orders with their OrderItem details and Product information from ES.
Used to build the Product mapping for Service_Charge__c enrichment.

## Filters Applied
- Order Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
- NOT PA MARKET DECOM
- Service_Order_Record_Type__c = 'Service Order Agreement'
- **Must have Billing_Invoice__c (BAN)** - required for migration

## Output
CSV file with Order, OrderItem, and Product2 details for mapping analysis.

In [2]:
# === SETUP & IMPORTS ===

import sys
import csv
from simple_salesforce import Salesforce
from datetime import datetime
from collections import Counter

print(f"Python: {sys.executable}")
print("✅ Set-up successful")

Python: /home/vjl2dev/miniconda3/envs/kairos-gpu/bin/python
✅ Set-up successful


In [ ]:
# === CONFIGURATION ===

# ES UAT Credentials
# ES_USERNAME = "sfdcapi@everstream.net.uat"
# ES_PASSWORD = "AS12df34!@#$"
# ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
# ES_DOMAIN = "test"

# # ES Production Credentials
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
ES_DOMAIN = "login"

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_active_orders_products_{timestamp}.csv"

print("📋 Configuration loaded")
print(f"   Output: {output_file}")

In [4]:
# === CONNECT TO ES ===

print("=" * 80)
print("CONNECTING TO ES SALESFORCE")
print("=" * 80)

es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

CONNECTING TO ES SALESFORCE
✅ Connected to ES: everstream.my.salesforce.com


In [5]:
# === QUERY ES ORDERITEMS WITH PRODUCT INFO ===
# Filters:
# - Active orders only (Activated, Suspended, Disconnect in Progress)
# - NOT PA MARKET DECOM
# - Service Order Agreement record type
# - MUST HAVE Billing_Invoice__c (BAN) - required for migration

print("\n" + "=" * 80)
print("QUERYING ES ORDERITEMS WITH PRODUCT INFO")
print("=" * 80)

query = """SELECT 
    Order.Id,
    Order.OrderNumber,
    Order.Name,
    Order.Status,
    Order.Billing_Invoice__c,
    Id,
    Total_MRC_Amortized__c,
    NRC_IRU_FEE__c,
    UnitPrice,
    TotalPrice,
    Product2.ProductCode,
    Product2.Family,
    Product2.Name
FROM OrderItem
WHERE Order.Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
AND (Order.Project_Group__c = null OR (NOT Order.Project_Group__c LIKE '%PA MARKET DECOM%'))
AND Order.Service_Order_Record_Type__c = 'Service Order Agreement'
AND Order.Billing_Invoice__c != null
ORDER BY Order.OrderNumber"""

print(f"Query:\n{query}")
print("\nExecuting query...")

result = es_sf.query_all(query)
records = result["records"]

print(f"\n✅ Found {len(records)} OrderItem records")


QUERYING ES ORDERITEMS WITH PRODUCT INFO
Query:
SELECT 
    Order.Id,
    Order.OrderNumber,
    Order.Name,
    Order.Status,
    Order.Billing_Invoice__c,
    Id,
    Total_MRC_Amortized__c,
    NRC_IRU_FEE__c,
    UnitPrice,
    TotalPrice,
    Product2.ProductCode,
    Product2.Family,
    Product2.Name
FROM OrderItem
WHERE Order.Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
AND (Order.Project_Group__c = null OR (NOT Order.Project_Group__c LIKE '%PA MARKET DECOM%'))
AND Order.Service_Order_Record_Type__c = 'Service Order Agreement'
AND Order.Billing_Invoice__c != null
ORDER BY Order.OrderNumber

Executing query...

✅ Found 22461 OrderItem records


In [5]:
# === EXPORT TO CSV ===

print("\n" + "=" * 80)
print("EXPORTING TO CSV")
print("=" * 80)

with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # Header
    writer.writerow(
        [
            "Order_Id",
            "Order_Number",
            "Order_Name",
            "Order_Status",
            "Billing_Invoice_Id",
            "OrderItem_Id",
            "MRC",
            "NRC",
            "UnitPrice",
            "TotalPrice",
            "Product_Code",
            "Product_Family",
            "Product_Name",
        ]
    )

    # Data
    for r in records:
        order = r.get("Order", {}) or {}
        product = r.get("Product2", {}) or {}
        writer.writerow(
            [
                order.get("Id", ""),
                order.get("OrderNumber", ""),
                order.get("Name", ""),
                order.get("Status", ""),
                order.get("Billing_Invoice__c", ""),
                r.get("Id", ""),
                r.get("Total_MRC_Amortized__c", ""),
                r.get("NRC_IRU_FEE__c", ""),
                r.get("UnitPrice", ""),
                r.get("TotalPrice", ""),
                product.get("ProductCode", ""),
                product.get("Family", ""),
                product.get("Name", ""),
            ]
        )

print(f"\n✅ Exported to: {output_file}")
print(f"   Total records: {len(records)}")


EXPORTING TO CSV

✅ Exported to: es_active_orders_products_20260114_133212.csv
   Total records: 22460


In [6]:
# === ANALYSIS: UNIQUE PRODUCTS AND FAMILIES ===

print("\n" + "=" * 80)
print("PRODUCT ANALYSIS")
print("=" * 80)

# Count product families
family_counts = Counter()
product_counts = Counter()

for r in records:
    product = r.get("Product2", {}) or {}
    family = product.get("Family") or "(No Family)"
    name = product.get("Name") or "(No Name)"
    family_counts[family] += 1
    product_counts[name] += 1

print(f"\n📊 Product Families ({len(family_counts)} unique):")
print("-" * 60)
for family, count in family_counts.most_common():
    print(f"  {count:>6}  {family}")

print(f"\n📊 Top 30 Products ({len(product_counts)} unique total):")
print("-" * 60)
for product, count in product_counts.most_common(30):
    print(f"  {count:>6}  {product[:50]}")

if len(product_counts) > 30:
    print(f"\n  ... and {len(product_counts) - 30} more products")


PRODUCT ANALYSIS

📊 Product Families (49 unique):
------------------------------------------------------------
    5338  Point-to-Point (PTPS)
    3607  Dedicated Internet Access (DIAS)
    3444  IP
    1823  Dark Fiber (DFBR)
    1703  Hosted Voice (VOIC)
    1539  Point-to-MultiPoint (PMPS)
     879  Promotions
     829  Managed Service (MSP)
     627  Diversity
     615  Handoff Type
     304  Tagged / Untagged
     297  Additional Port
     276  Logical Attributes
     152  Network-to-Network Interface (NNIS)
     126  Routing
     123  Managed Wave (MWAV)
      94  Collocation (COLO)
      77  TSP Fee
      63  Data Center Services (CLDS)
      63  Virtual Dedicated Internet Access (VDIA)
      61  Dark Fiber Pair IRU (DFBR)
      54  Ethernet (EPL/EVPL) (ETHS)
      43  Private Line (PVLS)
      40  Ethernet Transport Only (ETHS)
      36  Special Access Private Line (SAPL)
      30  Dedicated DWDM (DWDM)
      29  Power (COLO)
      25  Rack (COLO)
      19  PRI (SIP) (VOIC)
  

In [7]:
# === SUMMARY ===

print("\n" + "=" * 80)
print("EXPORT COMPLETE")
print("=" * 80)

# Count unique orders
unique_orders = set()
for r in records:
    order = r.get("Order", {}) or {}
    if order.get("Id"):
        unique_orders.add(order.get("Id"))

print(f"\n📈 Summary:")
print(f"   Unique Orders: {len(unique_orders)}")
print(f"   Total OrderItems: {len(records)}")
print(f"   Unique Product Families: {len(family_counts)}")
print(f"   Unique Products: {len(product_counts)}")
print(f"\n📁 Output file: {output_file}")
print(
    "\n✅ Use this data to build Product_Simple__c and Service_Type_Charge__c mappings"
)


EXPORT COMPLETE

📈 Summary:
   Unique Orders: 12594
   Total OrderItems: 22460
   Unique Product Families: 49
   Unique Products: 681

📁 Output file: es_active_orders_products_20260114_133212.csv

✅ Use this data to build Product_Simple__c and Service_Type_Charge__c mappings


---
## Next Steps: Product Mapping

Use the exported CSV to create a mapping from ES Product2 to BBF picklist values:

| ES Product Family | ES Product Name | BBF Product_Simple__c | BBF Service_Type_Charge__c |
|-------------------|-----------------|----------------------|---------------------------|
| Ethernet (EPL/EVPL) | 0100Mbps Ethernet Transport | ? | ? |
| Dark Fiber (DFBR) | Dark Fiber 1 Pair | ? | ? |
| ... | ... | ... | ... |

Once mapping is complete, use the enrichment process in `06_service_charge_migration.ipynb` to update records.